# Build Notes

In [ ]:
using Revise
using FUSE
using Plots;
FUSE.logging(Logging.Info);

## Initialize FPP v1_demount

In [ ]:
ini, act = FUSE.case_parameters(:FPP; version=:v1_demount, init_from=:scalars);

ini.build.layers[:hfs_vacuum_vessel] += ini.build.layers[:hfs_vacuum_vessel_wall_inner] + ini.build.layers[:hfs_vacuum_vessel_wall_outer]
delete!(ini.build.layers,:hfs_vacuum_vessel_wall_inner)
delete!(ini.build.layers,:hfs_vacuum_vessel_wall_outer)
ini.build.layers[:lfs_vacuum_vessel] += ini.build.layers[:lfs_vacuum_vessel_wall_inner] + ini.build.layers[:lfs_vacuum_vessel_wall_outer]
delete!(ini.build.layers,:lfs_vacuum_vessel_wall_inner)
delete!(ini.build.layers,:lfs_vacuum_vessel_wall_outer)
display(ini.build)
ini.equilibrium.B0 = 8.0
display(act.ActorHFSsizing)
act.ActorPFcoilsOpt.update_equilibrium = true



In [ ]:
dd = IMAS.dd()
FUSE.init(dd, ini, act; do_plot=false);
FUSE.digest(dd)

In [ ]:
println("OH R_start = ", dd.build.layer[2].start_radius)
println("OH R_end = ",dd.build.layer[2].end_radius)

println("TF R_start = ", dd.build.layer[4].start_radius)
println("TF R_end = ",dd.build.layer[4].end_radius)

dd.build.layer[8].material = "Steel, Stainless 316"
dd.build.layer[18].material = "Steel, Stainless 316"
dd.build.layer[24].material = "Steel, Stainless 316"


display(dd.build)

display(dd.build.tf)
display(dd.build.oh)

## Experiment with HFS build settings

In [ ]:
act.ActorHFSsizing.do_plot = true
act.ActorHFSsizing.verbose = true
act.ActorHFSsizing.aspect_ratio_tolerance = 0.01
act.ActorHFSsizing.j_tolerance = 0.5
act.ActorHFSsizing.stress_tolerance = 0.2
act.ActorHFSsizing.operate_at_j_crit = true
act.ActorHFSsizing

In [ ]:
FUSE.ActorHFSsizing(dd, act);

In [ ]:
FUSE.ActorLFSsizing(dd, act);
FUSE.ActorCXbuild(dd, act);
FUSE.ActorPFcoilsOpt(dd, act);
FUSE.ActorPassiveStructures(dd,act);


In [ ]:
using Printf

@printf("OH R_start = %.2f m\n", dd.build.layer[2].start_radius)
@printf("OH R_end   = %.2f m\n",dd.build.layer[2].end_radius)

@printf("TF R_start = %.2f m\n", dd.build.layer[4].start_radius)
@printf("TF R_end   = %.2f m\n",dd.build.layer[4].end_radius)

#display(dd.build.tf)
#display(dd.build.oh)

tf_j_max = dd.build.tf.max_j
oh_j_max = dd.build.oh.max_j

tf_j_crit = dd.build.tf.critical_j
oh_j_crit = dd.build.oh.critical_j

tf_vms_max = (maximum(abs.(dd.solid_mechanics.center_stack.stress.vonmises.tf)))
oh_vms_max = (maximum(abs.(dd.solid_mechanics.center_stack.stress.vonmises.oh)))


@printf("\n")
@printf("TF stainless_fraction = %.3f\n", dd.build.tf.technology.fraction_stainless)
@printf("OH stainless_fraction = %.3f\n", dd.build.oh.technology.fraction_stainless)

@printf("\n")
@printf("TF J_op = %.1f MA\n", tf_j_max/1e6)
#@printf("TF J_max = %.1f MA\n", tf_j_crit*(1-act.ActorHFSsizing.j_tolerance)/1e6)
@printf("TF J_crit = %.1f MA\n", tf_j_crit/1e6)

@printf("\n")
@printf("OH J_op = %.1f MA\n", oh_j_max/1e6)
#@printf("OH J_max = %.1f MA\n", oh_j_crit*(1-act.ActorHFSsizing.j_tolerance)/1e6)
@printf("OH J_crit = %.1f MA\n", oh_j_crit/1e6)

@printf("\n")
@printf("TF stress_op = %.1f MPa\n", tf_vms_max/1e6)
#@printf("TF stress_max = %.1f MPa\n", 800*(1-act.ActorHFSsizing.stress_tolerance))
@printf("TF stress_crit = %.1f MPa\n", 800)

@printf("\n")
@printf("OH stress_op = %.1f MPa\n", oh_vms_max/1e6)
#@printf("OH stress_max = %.1f MPa\n", 800*(1-act.ActorHFSsizing.stress_tolerance))
@printf("OH stress_crit = %.1f MPa\n", 800)




In [ ]:
plot(dd.solid_mechanics.center_stack.stress)

## Change CX build settings

In [ ]:
FUSE.digest(dd)